# 0. Google Colab Setup

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir('./drive/MyDrive/cmpt419 affective computing project')

In [ ]:
!pip install keras_pos_embd

  Preparing metadata (setup.py) ... done
  Created wheel for keras_pos_embd: filename=keras_pos_embd-0.13.0-py3-none-any.whl size=6945 sha256=f7d07e8c36048719514702c5e8aaa3f14724199275547f330a7055cc8456c261
  Stored in directory: /root/.cache/pip/wheels/78/07/1b/b1ca47b6ac338554b75c8f52c54e6a2bfbe1b07d79579979a4
Successfully built keras_pos_embd


In [ ]:
!pip install keras_layer_normalization

  Preparing metadata (setup.py) ... done
  Created wheel for keras_layer_normalization: filename=keras_layer_normalization-0.16.0-py3-none-any.whl size=4654 sha256=511125a9452eb3f3ae40e40e8d24c44fdefb7e314453bedd81000c7f03287251
  Stored in directory: /root/.cache/pip/wheels/ed/3a/4b/21db23c0cc56c4b219616e181f258eb7c57d36cc5d056fae9a
Successfully built keras_layer_normalization


In [ ]:
!pip install keras_multi_head

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for keras_multi_head: filename=keras_multi_head-0.29.0-py3-none-any.whl size=14976 sha256=b9bf3ac761064fa4d84880ce54e509905d2ba9db05dc2f60b55abc43598d76e6
  Stored in directory: /root/.cache/pip/wheels/cb/23/4b/06d7ae21714f70fcc25b48f972cc8e5e7f4b6b764a038b509d
  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18894 sha256=5ed3561d0cd02bbf77713ab9b049eafeb1e63662bdcd8f87f5b3bc45913110bc
  Stored in directory: /root/.cache/pip/wheels/b8/f7/24/607b483144fb9c47b4ba2c5fba6b68e54aeee2d5bf6c05302e
Successfully built keras_multi_head keras-self-attention


In [ ]:
!pip install keras_position_wise_feed_forward

  Preparing metadata (setup.py) ... done
  Created wheel for keras_position_wise_feed_forward: filename=keras_position_wise_feed_forward-0.8.0-py3-none-any.whl size=4969 sha256=830a687f76d29568b696ddbad5c2798fb5b907b4b0f2b020ce9ade95e31e686c
  Stored in directory: /root/.cache/pip/wheels/c1/6a/04/d1706a53b23b2cb5f9a0a76269bf87925daa1bca09eac01b21
Successfully built keras_position_wise_feed_forward


In [ ]:
!pip install keras_embed_sim

  Preparing metadata (setup.py) ... done
  Created wheel for keras_embed_sim: filename=keras_embed_sim-0.10.0-py3-none-any.whl size=3944 sha256=ec4d4b721bde595402acb764df057b4e3f375d5bd9f74e666edbc5c906fc1d86
  Stored in directory: /root/.cache/pip/wheels/82/32/c7/fd35d0d1b840a6c7cbd4343f808d10d0f7b87d271a4dbe796f
Successfully built keras_embed_sim


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, GlobalAveragePooling2D, Activation, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras.layers import Reshape, Permute, Multiply, Add, Concatenate
from tensorflow.keras.layers import Lambda
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
import numpy as np

# other keras related library
from keras_pos_embd import PositionEmbedding, TrigPosEmbedding
from keras_layer_normalization import LayerNormalization
from keras_multi_head import MultiHeadAttention
from keras_position_wise_feed_forward import FeedForward
from keras_embed_sim import EmbeddingRet, EmbeddingSim

import math
from tqdm import tqdm

# 1. Transformer Encoders
+ The transformer encoders takes in the discriminative AU embeddings as inputs. The component of the AU correlation contains multi-head attentions layer and a feed forward layer.

In [ ]:
def gelu(x):
    """An approximation of gelu.
    See: https://arxiv.org/pdf/1606.08415.pdf
    """
    return 0.5 * x * (1.0 + tf.math.tanh(math.sqrt(2.0 / math.pi) * (x + 0.044715 * x * x * x)))

def get_custom_objects():
    return {
        'gelu': gelu,
        'LayerNormalization': LayerNormalization,
        'MultiHeadAttention': MultiHeadAttention,
        'FeedForward': FeedForward,
        'TrigPosEmbedding': TrigPosEmbedding,
        'EmbeddingRet': EmbeddingRet,
        'EmbeddingSim': EmbeddingSim,
    }


def _wrap_layer(name,
        input_layer,
        build_func,
        dropout_rate=0.0,
        trainable=True):
    """Wrap layers with residual, normalization and dropout.
    :param name: Prefix of names for internal layers.
    :param input_layer: Input layer.
    :param build_func: A callable that takes the input tensor and generates the output tensor.
    :param dropout_rate: Dropout rate.
    :param trainable: Whether the layers are trainable.
    :return: Output layer.
    """
    build_output = build_func(input_layer)
    if dropout_rate > 0.0:
        dropout_layer = Dropout(
            rate=dropout_rate,
            name='%s-Dropout' % name,
        )(build_output)
    else:
        dropout_layer = build_output
    if isinstance(input_layer, list):
        input_layer = input_layer[0]
    add_layer = Add(name='%s-Add' % name)([input_layer, dropout_layer])
    normal_layer = LayerNormalization(
        trainable=trainable,
        name='%s-Norm' % name,
    )(add_layer)
    return normal_layer

def attention_builder(name,
                      head_num,
                      activation,
                      history_only,
                      trainable=True):
    """Get multi-head self-attention builder.
    :param name: Prefix of names for internal layers.
    :param head_num: Number of heads in multi-head self-attention.
    :param activation: Activation for multi-head self-attention.
    :param history_only: Only use history data.
    :param trainable: Whether the layer is trainable.
    :return:
    """
    def _attention_builder(x):
        return MultiHeadAttention(
            head_num=head_num,
            activation=activation,
            history_only=history_only,
            trainable=trainable,
            name=name,
        )(x)
    return _attention_builder


def feed_forward_builder(name,
                         hidden_dim,
                         activation,
                         trainable=True):
    """Get position-wise feed-forward layer builder.
    :param name: Prefix of names for internal layers.
    :param hidden_dim: Hidden dimension of feed forward layer.
    :param activation: Activation for feed-forward layer.
    :param trainable: Whether the layer is trainable.
    :return:
    """
    def _feed_forward_builder(x):
        return FeedForward(
            units=hidden_dim,
            activation=activation,
            trainable=trainable,
            name=name,
        )(x)
    return _feed_forward_builder


def get_encoder_component(name,
              input_layer,
              head_num,
              hidden_dim,
              attention_activation=None,
              feed_forward_activation=gelu,
              dropout_rate=0.0,
              trainable=True,):
    """Multi-head self-attention and feed-forward layer.
    :param name: Prefix of names for internal layers.
    :param input_layer: Input layer.
    :param head_num: Number of heads in multi-head self-attention.
    :param hidden_dim: Hidden dimension of feed forward layer.
    :param attention_activation: Activation for multi-head self-attention.
    :param feed_forward_activation: Activation for feed-forward layer.
    :param dropout_rate: Dropout rate.
    :param trainable: Whether the layers are trainable.
    :return: Output layer.
    """
    attention_name = '%s-MultiHeadSelfAttention' % name
    feed_forward_name = '%s-FeedForward' % name
    attention_layer = _wrap_layer(
        name=attention_name,
        input_layer=input_layer,
        build_func=attention_builder(
            name=attention_name,
            head_num=head_num,
            activation=attention_activation,
            history_only=False,
            trainable=trainable,
        ),
        dropout_rate=dropout_rate,
        trainable=trainable,
    )
    feed_forward_layer = _wrap_layer(
        name=feed_forward_name,
        input_layer=attention_layer,
        build_func=feed_forward_builder(
            name=feed_forward_name,
            hidden_dim=hidden_dim,
            activation=feed_forward_activation,
            trainable=trainable,
        ),
        dropout_rate=dropout_rate,
        trainable=trainable,
    )

    return feed_forward_layer

# we didn't use the decoder
def get_decoder_component(name,
              input_layer,
              encoded_layer,
              head_num,
              hidden_dim,
              attention_activation=None,
              feed_forward_activation=gelu,
              dropout_rate=0.0,
              trainable=True):
    """Multi-head self-attention, multi-head query attention and feed-forward layer.
    :param name: Prefix of names for internal layers.
    :param input_layer: Input layer.
    :param encoded_layer: Encoded layer from encoder.
    :param head_num: Number of heads in multi-head self-attention.
    :param hidden_dim: Hidden dimension of feed forward layer.
    :param attention_activation: Activation for multi-head self-attention.
    :param feed_forward_activation: Activation for feed-forward layer.
    :param dropout_rate: Dropout rate.
    :param trainable: Whether the layers are trainable.
    :return: Output layer.
    """
    self_attention_name = '%s-MultiHeadSelfAttention' % name
    query_attention_name = '%s-MultiHeadQueryAttention' % name
    feed_forward_name = '%s-FeedForward' % name
    self_attention_layer = _wrap_layer(
        name=self_attention_name,
        input_layer=input_layer,
        build_func=attention_builder(
            name=self_attention_name,
            head_num=head_num,
            activation=attention_activation,
            history_only=True,
            trainable=trainable,
        ),
        dropout_rate=dropout_rate,
        trainable=trainable,
    )
    query_attention_layer = _wrap_layer(
        name=query_attention_name,
        input_layer=[self_attention_layer, encoded_layer, encoded_layer],
        build_func=attention_builder(
            name=query_attention_name,
            head_num=head_num,
            activation=attention_activation,
            history_only=False,
            trainable=trainable,
        ),
        dropout_rate=dropout_rate,
        trainable=trainable,
    )
    feed_forward_layer = _wrap_layer(
        name=feed_forward_name,
        input_layer=query_attention_layer,
        build_func=feed_forward_builder(
            name=feed_forward_name,
            hidden_dim=hidden_dim,
            activation=feed_forward_activation,
            trainable=trainable,
        ),
        dropout_rate=dropout_rate,
        trainable=trainable,
    )
    return feed_forward_layer

def get_encoders(name,encoder_num,
          input_layer,
          head_num,
          hidden_dim,
          attention_activation=None,
          feed_forward_activation=gelu,
          dropout_rate=0.0,
          trainable=True):
    """Get encoders.
    :param encoder_num: Number of encoder components.
    :param input_layer: Input layer.
    :param head_num: Number of heads in multi-head self-attention.
    :param hidden_dim: Hidden dimension of feed forward layer.
    :param attention_activation: Activation for multi-head self-attention.
    :param feed_forward_activation: Activation for feed-forward layer.
    :param dropout_rate: Dropout rate.
    :param trainable: Whether the layers are trainable.
    :return: Output layer.
    """
    last_layer = input_layer
    for i in range(encoder_num):
        last_layer = get_encoder_component(
            name='%sEncoder-%d' % (name,i + 1),
            input_layer=last_layer,
            head_num=head_num,
            hidden_dim=hidden_dim,
            attention_activation=attention_activation,
            feed_forward_activation=feed_forward_activation,
            dropout_rate=dropout_rate,
            trainable=trainable,
        )
    return last_layer


def get_decoders(name,decoder_num,
                 input_layer,
                 encoded_layer,
                 head_num,
                 hidden_dim,
                 attention_activation=None,
                 feed_forward_activation=gelu,
                 dropout_rate=0.0,
                 trainable=True):
    """Get decoders.
    :param decoder_num: Number of decoder components.
    :param input_layer: Input layer.
    :param encoded_layer: Encoded layer from encoder.
    :param head_num: Number of heads in multi-head self-attention.
    :param hidden_dim: Hidden dimension of feed forward layer.
    :param attention_activation: Activation for multi-head self-attention.
    :param feed_forward_activation: Activation for feed-forward layer.
    :param dropout_rate: Dropout rate.
    :param trainable: Whether the layers are trainable.
    :return: Output layer.
    """
    last_layer = input_layer
    for i in range(decoder_num):
        last_layer = get_decoder_component(
            name='%sDecoder-%d' % (name,i + 1),
            input_layer=last_layer,
            encoded_layer=encoded_layer,
            head_num=head_num,
            hidden_dim=hidden_dim,
            attention_activation=attention_activation,
            feed_forward_activation=feed_forward_activation,
            dropout_rate=dropout_rate,
            trainable=trainable,
        )
    return last_layer


# def get_model(token_num,
#               embed_dim,
#               encoder_num,
#               decoder_num,
#               head_num,
#               hidden_dim,
#               attention_activation=None,
#               feed_forward_activation=gelu,
#               dropout_rate=0.0,
#               use_same_embed=True,
#               embed_weights=None,
#               embed_trainable=None,
#               trainable=True):
#     """Get full model without compilation.
#     :param token_num: Number of distinct tokens.
#     :param embed_dim: Dimension of token embedding.
#     :param encoder_num: Number of encoder components.
#     :param decoder_num: Number of decoder components.
#     :param head_num: Number of heads in multi-head self-attention.
#     :param hidden_dim: Hidden dimension of feed forward layer.
#     :param attention_activation: Activation for multi-head self-attention.
#     :param feed_forward_activation: Activation for feed-forward layer.
#     :param dropout_rate: Dropout rate.
#     :param use_same_embed: Whether to use the same token embedding layer. `token_num`, `embed_weights` and
#                            `embed_trainable` should be lists of two elements if it is False.
#     :param embed_weights: Initial weights of token embedding.
#     :param embed_trainable: Whether the token embedding is trainable. It will automatically set to False if the given
#                             value is None when embedding weights has been provided.
#     :param trainable: Whether the layers are trainable.
#     :return: Keras model.
#     """
#     if not isinstance(token_num, list):
#         token_num = [token_num, token_num]
#     encoder_token_num, decoder_token_num = token_num

#     if not isinstance(embed_weights, list):
#         embed_weights = [embed_weights, embed_weights]
#     encoder_embed_weights, decoder_embed_weights = embed_weights
#     if encoder_embed_weights is not None:
#         encoder_embed_weights = [encoder_embed_weights]
#     if decoder_embed_weights is not None:
#         decoder_embed_weights = [decoder_embed_weights]

#     if not isinstance(embed_trainable, list):
#         embed_trainable = [embed_trainable, embed_trainable]
#     encoder_embed_trainable, decoder_embed_trainable = embed_trainable
#     if encoder_embed_trainable is None:
#         encoder_embed_trainable = encoder_embed_weights is None
#     if decoder_embed_trainable is None:
#         decoder_embed_trainable = decoder_embed_weights is None

#     if use_same_embed:
#         encoder_embed_layer = decoder_embed_layer = EmbeddingRet(
#             input_dim=encoder_token_num,
#             output_dim=embed_dim,
#             mask_zero=True,
#             weights=encoder_embed_weights,
#             trainable=encoder_embed_trainable,
#             name='Token-Embedding',
#         )
#     else:
#         encoder_embed_layer = EmbeddingRet(
#             input_dim=encoder_token_num,
#             output_dim=embed_dim,
#             mask_zero=True,
#             weights=encoder_embed_weights,
#             trainable=encoder_embed_trainable,
#             name='Encoder-Token-Embedding',
#         )
#         decoder_embed_layer = EmbeddingRet(
#             input_dim=decoder_token_num,
#             output_dim=embed_dim,
#             mask_zero=True,
#             weights=decoder_embed_weights,
#             trainable=decoder_embed_trainable,
#             name='Decoder-Token-Embedding',
#         )
#     encoder_input = Input(shape=(None,), name='Encoder-Input')
#     encoder_embed = TrigPosEmbedding(
#         mode=TrigPosEmbedding.MODE_ADD,
#         name='Encoder-Embedding',
#     )(encoder_embed_layer(encoder_input)[0])
#     encoded_layer = get_encoders(
#         encoder_num=encoder_num,
#         input_layer=encoder_embed,
#         head_num=head_num,
#         hidden_dim=hidden_dim,
#         attention_activation=attention_activation,
#         feed_forward_activation=feed_forward_activation,
#         dropout_rate=dropout_rate,
#         trainable=trainable,
#     )
#     decoder_input = Input(shape=(None,), name='Decoder-Input')
#     decoder_embed, decoder_embed_weights = decoder_embed_layer(decoder_input)
#     decoder_embed = TrigPosEmbedding(
#         mode=TrigPosEmbedding.MODE_ADD,
#         name='Decoder-Embedding',
#     )(decoder_embed)
#     decoded_layer = get_decoders(
#         decoder_num=decoder_num,
#         input_layer=decoder_embed,
#         encoded_layer=encoded_layer,
#         head_num=head_num,
#         hidden_dim=hidden_dim,
#         attention_activation=attention_activation,
#         feed_forward_activation=feed_forward_activation,
#         dropout_rate=dropout_rate,
#         trainable=trainable,
#     )
#     output_layer = EmbeddingSim(
#         trainable=trainable,
#         name='Decoder-Output',
#     )([decoded_layer, decoder_embed_weights])
#     return keras.models.Model(inputs=[encoder_input, decoder_input], outputs=output_layer)

def _get_max_suffix_repeat_times(tokens, max_len):
    detect_len = min(max_len, len(tokens))
    next = [-1] * detect_len
    k = -1
    for i in range(1, detect_len):
        while k >= 0 and tokens[len(tokens) - i - 1] != tokens[len(tokens) - k - 2]:
            k = next[k]
        if tokens[len(tokens) - i - 1] == tokens[len(tokens) - k - 2]:
            k += 1
        next[i] = k
    max_repeat = 1
    for i in range(2, detect_len):
        if next[i] >= 0 and (i + 1) % (i - next[i]) == 0:
            max_repeat = max(max_repeat, (i + 1) // (i - next[i]))
    return max_repeat

def decode(model,
      tokens,
      start_token,
      end_token,
      pad_token,
      top_k=1,
      temperature=1.0,
      max_len=10000,
      max_repeat=10,
      max_repeat_block=10):
    """Decode with the given model and input tokens.
    :param model: The trained model.
    :param tokens: The input tokens of encoder.
    :param start_token: The token that represents the start of a sentence.
    :param end_token: The token that represents the end of a sentence.
    :param pad_token: The token that represents padding.
    :param top_k: Choose the last token from top K.
    :param temperature: Randomness in boltzmann distribution.
    :param max_len: Maximum length of decoded list.
    :param max_repeat: Maximum number of repeating blocks.
    :param max_repeat_block: Maximum length of the repeating block.
    :return: Decoded tokens.
    """
    is_single = not isinstance(tokens[0], list)
    if is_single:
        tokens = [tokens]
    batch_size = len(tokens)
    decoder_inputs = [[start_token] for _ in range(batch_size)]
    outputs = [None for _ in range(batch_size)]
    output_len = 1
    while len(list(filter(lambda x: x is None, outputs))) > 0:
        output_len += 1
        batch_inputs, batch_outputs = [], []
        max_input_len = 0
        index_map = {}
        for i in range(batch_size):
            if outputs[i] is None:
                index_map[len(batch_inputs)] = i
                batch_inputs.append(tokens[i][:])
                batch_outputs.append(decoder_inputs[i])
                max_input_len = max(max_input_len, len(tokens[i]))
        for i in range(len(batch_inputs)):
            batch_inputs[i] += [pad_token] * (max_input_len - len(batch_inputs[i]))
        predicts = model.predict([np.array(batch_inputs), np.array(batch_outputs)])
        for i in range(len(predicts)):
            if top_k == 1:
                last_token = predicts[i][-1].argmax(axis=-1)
            else:
                probs = [(prob, j) for j, prob in enumerate(predicts[i][-1])]
                probs.sort(reverse=True)
                probs = probs[:top_k]
                indices, probs = list(map(lambda x: x[1], probs)), list(map(lambda x: x[0], probs))
                probs = np.array(probs) / temperature
                probs = probs - np.max(probs)
                probs = np.exp(probs)
                probs = probs / np.sum(probs)
                last_token = np.random.choice(indices, p=probs)
            decoder_inputs[index_map[i]].append(last_token)
            if last_token == end_token or\
                    (max_len is not None and output_len >= max_len) or\
                    _get_max_suffix_repeat_times(decoder_inputs[index_map[i]],
                                                 max_repeat * max_repeat_block) >= max_repeat:
                outputs[index_map[i]] = decoder_inputs[index_map[i]]
    if is_single:
        outputs = outputs[0]
    return outputs

# 2. Self-defined Losses

In [ ]:
def get_centers(y_true,y_pred,centers):
        output_list = []
        for i in range(AU_count):
            labels = tf.reshape(y_true[:,i],[-1])
            labels = tf.cast(labels,tf.float32)
            # masked_features = centers[i,...]+y_pred[:,i,:]*labels[:,tf.newaxis]
            masked_features = centers[i, ...] + y_pred[:, i, :] * tf.expand_dims(labels, axis=-1)
            moving_average = tf.math.reduce_mean(masked_features,axis=0)
            output_list.append(moving_average)
        centers = tf.stack(output_list)
        return centers

def get_center_loss(alpha,y_true,y_pred,centers):
    loss=0
    for i in range(AU_count):
        labels = tf.reshape(y_true[:,i],[-1])
        labels = tf.cast(labels,tf.float32)
        # diff = alpha[i]*(y_pred[:,i,:]-centers[i,...])*labels[:,tf.newaxis]
        diff = alpha[i] * (y_pred[:, i, :] - centers[i, ...]) * tf.expand_dims(labels, axis=-1)
        loss += tf.reduce_mean(tf.square(diff))
    return loss

def get_divergence_loss(alpha,y_true,y_pred,centers):
    loss=0
    for i in range(AU_count):
        labels = tf.reshape(1-y_true[:,i],[-1])
        labels = tf.cast(labels,tf.float32)
        # diff = alpha[i]*(y_pred[:,i,:]-centers[i,...])*labels[:,tf.newaxis]
        diff = alpha[i] * (y_pred[:, i, :] - centers[i, ...]) * tf.expand_dims(labels, axis=-1)
        loss += tf.reduce_mean(tf.square(diff))
    return loss

# def get_full_center_loss(alpha, feature_dim,ind):
#     # Build a graph
#     graph = tf.Graph()
#     with graph.as_default():
#         centers = tf.compat.v1.get_variable('centers_{}'.format(ind), shape=[AU_count,256],
#                             initializer=tf.compat.v1.constant_initializer(0),
#                             dtype=tf.float32)     # Create a variable tensor

#     # Create a session, and run the graph
#     with tf.compat.v1.Session(graph=graph) as session:
#         tf.compat.v1.global_variables_initializer().run()  # Initialize values of all variable tensors
#         output_a = session.run(centers)            # Return the value of the variable tensor

#     centers = output_a
#     # print(centers)
#     def center_loss(y_true, y_pred):
# #         a = np.zeros([AU_count, feature_dim])
#         c = tf.constant(centers)
#         c = get_centers(y_true,y_pred,c)
# #         centers = c
# #         centers.assign(c)
#         num_loss = get_center_loss(alpha,y_true,y_pred,c)
#         den_loss = get_divergence_loss(alpha,y_true,y_pred,c)
#         # print("full num_loss return shape: ", den_loss.shape)
#         # print("full den_loss return shape: ", num_loss.shape)
#         # print("full center loss return shape: ", (num_loss/den_loss).shape)
#         return num_loss/den_loss

#     return center_loss

# rewrite the get_full_center_loss in tensorflow 2.15 style
def get_full_center_loss(alpha, feature_dim, ind):
  centers = tf.Variable(tf.zeros([AU_count, 256]), name='centers_{}'.format(ind), dtype=tf.float32)
  def center_loss(y_true, y_pred):
      c = get_centers(y_true, y_pred, centers)
      num_loss = get_center_loss(alpha, y_true, y_pred, c)
      den_loss = get_divergence_loss(alpha, y_true, y_pred, c)
      return num_loss / den_loss
  return center_loss

# the weighted_bce loss is depricated 
def weighted_bce(y_true, y_pred,weights):
    # weights = ((y_true) * 59.) + 1.
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred,from_logits=False)
    weighted_bce = tf.math.reduce_mean(bce * weights)
    return weighted_bce

def macro_soft_f1(weights):
    def loss(y, y_hat):
        """Compute the macro soft F1-score as a cost (average 1 - soft-F1 across all labels).
        Use probability values instead of binary predictions.
        Args:
            y (int32 Tensor): targets array of shape (BATCH_SIZE, N_LABELS)
            y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        Returns:
            cost (scalar Tensor): value of the cost function for the batch
        """
        # print(weights)
        # ensure data type
        y = tf.cast(y, tf.float32)
        y_hat = tf.cast(y_hat, tf.float32)
        # calculate component for soft f1
        tp = tf.math.reduce_sum(y_hat * y, axis=0)
        fp = tf.math.reduce_sum(y_hat * (1 - y), axis=0)
        fn = tf.math.reduce_sum((1 - y_hat) * y, axis=0)
        tn = tf.math.reduce_sum((1 - y_hat) * (1-y), axis=0)
        # soft f1 is calculated by F1 = (tp/(tp+0.25fn+0.75fp))
        soft_f1 = (tp+ 1e-16) / (tp + 0.25*fn + 0.75*fp + 1e-16)
        # the lower the soft f1 the higher the cost
        cost = 1 - soft_f1 # reduce 1 - soft-f1 in order to increase soft-f1
        macro_cost = tf.math.reduce_mean(cost*weights)+weighted_bce(y,y_hat,weights) # +average on all labels
        return macro_cost
    return loss

# for attention map loss calculation
def huber_loss(y_true, y_pred, clip_delta=0.1):
    error = y_true - y_pred
    cond  = tf.math.abs(error) < clip_delta
    squared_loss = 0.5 * tf.math.square(error)
    linear_loss  = clip_delta * (tf.math.abs(error) - 0.5 * clip_delta)
    return tf.where(cond, squared_loss, linear_loss)

def macro_f1(y, y_hat, thresh=0.5):
    """Compute the macro F1-score on a batch of observations (average F1 across labels)
    Args:
        y (int32 Tensor): labels array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        thresh: probability value above which we predict positive
    Returns:
        macro_f1 (scalar Tensor): value of macro F1 for the batch
    """
    y_pred = tf.cast(tf.math.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = (2*tp+1e-16) / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.math.reduce_mean(f1)#+tf.keras.losses.binary_crossentropy(y,y_hat)
    return macro_f1

# 3. Load Pretrained Model
+ The pretrained model is trained and saved using tf==1.15, but it is still usable after changing the code to tf==2.15 style.

In [ ]:
# use InceptionV3 as the feature extracter
base_model0 = InceptionV3(weights="imagenet",include_top=False, input_shape= (224,224,3))
base_model = Model(inputs=base_model0.input,outputs = base_model0.get_layer('activation_74').output)

In [ ]:
AU_count=12
data_path = './DISFA_fold/'
ind1 = [0, 1, 2, 3, 4, 6, 8, 10, 11, 12, 15, 16]

In [ ]:
def batch_generator(x):
    attsize=12
    i=0
    ind = ind1
    while True:
      sample_idx = 0
      batch = 0
      print('entering ... ',len(x))
      while sample_idx  < len(x):
        l = np.load(data_path+x[sample_idx]) # load an npz file with x_train, y_train, x_train_att
        x_train1 = l['x_train1']
        x_train1 = np.concatenate((x_train1,x_train1,x_train1),axis=-1) # (224,224,1) -> (224, 224, 3)
        x_train_att = l['x_train_att']
        x_train_att = np.resize(x_train_att,(64,attsize,attsize,18))
        y_train1 = l['y_train']
        x_train1_att = x_train_att[...,ind]/255.
        y_train=y_train1[:,ind]
        y_train1 = np.expand_dims(y_train,axis=-2)
        y_train2 = y_train
        sample_idx += 1
        batch+=1
        # print("att_outputs: ", y_train.shape)
        # print("per_output: ", y_train.shape)
        # print("att_loss:", x_train1_att.shape)
        # print("feat_outputs: ", y_train.shape)
        yield(x_train1, {"att_outputs":y_train,"per_outputs_{}".format(AU_count): y_train, "att_loss":x_train1_att,"feat_outputs_{}".format(AU_count):y_train})

In [ ]:
# need to create image list file
img_list = [] # store the name of each image("SN001_frame_28.jpg")
img_path_list = [] # store each image's path
IMAGE_PATH = './dataset/frames'
for subject in os.listdir(IMAGE_PATH):
  subject_path = os.path.join(IMAGE_PATH, subject)
  for frame in os.listdir(subject_path):
    img_list.append("{}_{}".format(subject, frame))
    curr_path = os.path.join(subject_path, frame)
    img_path_list.append(curr_path)

In [ ]:
fold1 = ['SN002','SN010','SN001','SN026','SN027','SN030','SN032','SN009','SN016']
fold2 = ['SN006','SN011','SN012','SN013','SN018','SN021','SN024','SN028','SN031']
fold3 = ['SN003','SN004','SN005','SN007','SN008','SN017','SN023','SN025','SN029']

Total_att_acc =[];Total_perc_acc=[];Total_att_f1 =[];Total_perc_f1=[]
index_list = [] # store the index = 4845 * (subject_idx-1) + frame - 1
index_im_list = [] # store the img_path

images = img_list

for _ in range(3):
    index_list.append([])
    index_im_list.append([])
for im in range(len(images)):
    subject = images[im].split('_')[0]
    subject_idx = int(subject[2:])
    img_idx = 4845 * (subject_idx-1) + int(images[im].split('_')[2][:-4]) - 1
    if subject in fold1:
        index_list[0].append(img_idx);index_im_list[0].append(img_path_list[im])
    elif subject in fold2:
        index_list[1].append(img_idx);index_im_list[1].append(img_path_list[im])
    elif subject in fold3:
        index_list[2].append(img_idx);index_im_list[2].append(img_path_list[im])

In [ ]:
# use fold1 and fold2 as the training set, and keep fold0 as the test set 
dataset = 'DISFA'

ind = 0
# use one fold as validation set and the resting two folds as the training set
x_test = index_im_list[ind]
new = list(range(3))
new.remove(ind)

# load the processed data
batches_test = [f for f in os.listdir(data_path) if f.startswith(dataset+'_fold'+str(ind))]

batches_train = [f for f in os.listdir(data_path) if f.startswith(dataset+'_fold'+str(new[0]))]

batches_train += [f for f in os.listdir(data_path) if f.startswith(dataset+'_fold'+str(new[1]))]

y_train = np.zeros((len(batches_train)*64,AU_count ), dtype='float32')
row=0
for batch_iter in tqdm(batches_train):
    t_t=np.load(data_path+batch_iter)['y_train'][:,ind1] # extract the au exisit in DISFA dataset
    y_train[row:row+t_t.shape[0],...]=t_t
    row+=t_t.shape[0]

100%|██████████| 1122/1122 [13:44<00:00,  1.36it/s]


In [ ]:
# Initialize alpha and weights(here weights are used for weighted loss generation)
tp = np.sum(y_train,axis=0)
fp = np.sum((1-y_train),axis=0)
Pni = (tp+1e-6)/(tp+fp+1e-6)
alpha = 1/(AU_count*Pni)
Psi = np.power(alpha,0.5)*Pni
weights = np.sum(Psi)/Psi

In [ ]:
def baseline_model_best(AU_count,weights,ind):
  fc_dim = 256
  inputs = Input(shape=(224,224,3))

  #block 1
  g = base_model(inputs)

  gh = Conv2D(64, (3,3), padding='same', kernel_initializer='glorot_normal')(g)
  gh1 = Conv2D(AU_count, (1,1), padding='same', kernel_initializer='glorot_normal')(gh)
  gh2 = Conv2D(AU_count, (1,1), padding='same', activation='sigmoid',name = "att_loss", kernel_initializer='glorot_normal')(gh1)
  gh1 = Conv2D(AU_count, (1,1), padding='same', activation='linear', kernel_initializer='glorot_normal')(gh1)
  gap = GlobalAveragePooling2D()(gh1)
  att_output = Activation('sigmoid',name="att_outputs")(gap)
  attention = gh2
  reshape_embed = Reshape([12*12,AU_count])(attention)
  reshape_embed = Permute((2,1))(reshape_embed)
  print("in model attention shape: ", attention.shape)

  # AU attention layers
  for i in range(AU_count):

      # layer1 = Lambda(lambda x: tf.expand_dims(attention[...,i],axis=-1))(attention)
      layer1 = tf.expand_dims(attention[..., i], axis=-1)

      out = Multiply()([layer1,g])
      g = Add()([out,g])
      mt = Conv2D(64, (1,1), padding='same', kernel_initializer='glorot_normal')(g)
      mt = MaxPooling2D(pool_size=7, strides=(1,1),padding = 'same')(mt)
      mt = BatchNormalization()(mt)
      mt = Activation('relu')(mt)
      perception = Flatten()(mt)

      inter = Dense(fc_dim, activation='relu', kernel_initializer='glorot_normal')(perception)
      # tin = Lambda(lambda x: tf.expand_dims(x,axis=1))(inter)
      tin = tf.expand_dims(inter, axis=1)
      if i==0:
          feat_outputs = tin
      else:
          feat_outputs = Concatenate(axis = 1, name = 'feat_outputs_{}'.format(i+1))([feat_outputs,tin])

  feat_outputs_P  = PositionEmbedding(
      input_shape=(None,),
      input_dim = AU_count,
      output_dim = fc_dim,
      mask_zero=0,
      mode=PositionEmbedding.MODE_ADD,)(feat_outputs)
  feat_outputs_P = get_encoders(name = '1',encoder_num=3,
                  input_layer=feat_outputs_P,
                  head_num=8,hidden_dim=fc_dim,
                  dropout_rate=0.1,)

  feat_outputs_P = Flatten()(feat_outputs_P)
  inter = Dense(fc_dim, activation='relu', kernel_initializer='glorot_normal')(feat_outputs_P)
  final = Dense(AU_count,
          activation='sigmoid',
          name = 'per_outputs_{}'.format(AU_count),
          kernel_initializer='glorot_normal')(inter)
  model = Model(inputs=inputs, outputs=[att_output,final,attention,feat_outputs])
  # model.summary()

  # losses = {
  #     "att_outputs": macro_soft_f1(weights),
  #     "per_outputs_{}".format(i+1): macro_soft_f1(weights),
  #     "att_loss": huber_loss,
  #     'feat_outputs_{}'.format(i+1):get_full_center_loss(weights,fc_dim,ind)
  #     }

  losses = {
      "att_outputs": "binary_crossentropy",
      "per_outputs_{}".format(AU_count): "binary_crossentropy",
      "att_loss": huber_loss,
      'feat_outputs_{}'.format(AU_count):get_full_center_loss(weights,fc_dim,ind)
      }

  lossWeights = {"att_outputs": 0.25,
          "per_outputs_{}".format(AU_count): 0.25,
          "att_loss":0.25,
          'feat_outputs_{}'.format(AU_count):0.25}
  
  # initialize the optimizer and compile the model
  print("[INFO] compiling model...")
  opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
  model.compile(optimizer=opt,
          loss=losses,
          loss_weights=lossWeights,
          metrics={'att_outputs':macro_f1,
          "per_outputs_{}".format(AU_count):macro_f1,
          "att_loss":huber_loss})
  return model

In [ ]:
model = baseline_model_best(AU_count,weights*0.1,ind)
if ind==-1: # here we don't train from scratch thus set it to -1
    model.save_weights('initial.h5')
else: # use model pretrained on BP4D dataset and finetune it on DISFA dataset
    model.load_weights('Pretrianed_Transformer_FAU_fold0.h5')
# initialize the optimizer and compile the model
reduce_lr = ReduceLROnPlateau(monitor='val_per_outputs_12_macro_f1',
                              mode='max',factor=0.2,
                              patience=3, min_lr=0.0000001)
mcp_save = ModelCheckpoint('models/Transformer_FAU_fold{}_newversion.h5'.format(ind),
              save_best_only=True, save_weights_only=True,
              monitor='val_att_outputs_macro_f1', mode='max')
if ind>=0:
  # use fit_generator to load data with batch in case the memeroy does not fit
  trainer = model.fit_generator(batch_generator(batches_train),
            validation_data= batch_generator(batches_test),
            epochs = 1,
            steps_per_epoch=len(batches_train) ,
            validation_steps=len(batches_test),
            callbacks=[reduce_lr,mcp_save])

in model attention shape:  (None, 12, 12, 12)


/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


[INFO] compiling model...


<ipython-input-22-7f6a865e03a2>:15: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  trainer = model.fit_generator(batch_generator(batches_train),


entering ...  1122
1122/1122 [==============================] - ETA: 0s - loss: 0.3282 - att_outputs_loss: 0.9187 - per_outputs_12_loss: 0.3599 - att_loss_loss: 0.0343 - feat_outputs_12_loss: 2.9109e-10 - att_outputs_macro_f1: 0.4184 - per_outputs_12_macro_f1: 0.7316 - att_loss_huber_loss: 0.0343entering ...  378
entering ...  378
1122/1122 [==============================] - 1085s 907ms/step - loss: 0.3282 - att_outputs_loss: 0.9187 - per_outputs_12_loss: 0.3599 - att_loss_loss: 0.0343 - feat_outputs_12_loss: 2.9109e-10 - att_outputs_macro_f1: 0.4184 - per_outputs_12_macro_f1: 0.7316 - att_loss_huber_loss: 0.0343 - val_loss: 1.0279 - val_att_outputs_loss: 0.6831 - val_per_outputs_12_loss: 3.0810 - val_att_loss_loss: 0.0338 - val_feat_outputs_12_loss: 0.3139 - val_att_outputs_macro_f1: 0.5313 - val_per_outputs_12_macro_f1: 0.6831 - val_att_loss_huber_loss: 0.0338 - lr: 1.0000e-05


# 4. Model Inference
+ Use the finetuned model for test set inference and check its F1 score on each exisiting AU

In [ ]:
# predict one batch of testing data
y_pred_acc = []
y_pred_f1 = np.array([])
y_true_ex = []

data_test_fold0 = './DISFA_fold/DISFA_fold0_10059.npz'
# model.load_weights('Transformer_FAU_fold0.h5')
print('Predicting '+ data_test_fold0)
x = np.load(data_test_fold0)
x_train1 = x['x_train1']
x_train1 = np.concatenate((x_train1,x_train1,x_train1),axis=-1)
y_predict = model.predict(x_train1)
y_test = x['y_train']
y_test=y_test[:,ind1]

y_pred = y_predict[1]
t = y_pred[2]
print(y_pred.shape)
y_pred_acc.append(y_pred)
y_true = y_test>0.5

y_true_ex.append(y_true)

y_pred_acc = np.vstack(y_pred_acc)
y_true_ex = np.vstack(y_true_ex)
att_f1=[]
for l in range(AU_count):
    a = np.multiply(y_true_ex[:,l],1)
    b = np.multiply(y_pred_acc[:,l]>0.5,1)
    TP = sum(np.multiply(np.logical_and(a==1,b==1),1))
    FP = sum(np.multiply(np.logical_and(a==0,b==1),1))
    FN = sum(np.multiply(np.logical_and(a==1,b==0),1))
    TN = sum(np.multiply(np.logical_and(a==0,b==0),1))
    Precision = TP/(TP+FP+0.00001)
    print('TP',TP,'FP',FP,'FN',FN,'TN',TN)
    Recall = TP/(TP+FN+0.00001)
    att_f1.append(2*Precision*Recall/(Precision+Recall+0.000001))

print(' f1 score ....', att_f1)

Predicting ./DISFA_fold/DISFA_fold0_10059.npz
2/2 [==============================] - 0s 92ms/step
(64, 12)
TP 0 FP 0 FN 0 TN 64
TP 0 FP 0 FN 0 TN 64
TP 0 FP 0 FN 0 TN 64
TP 0 FP 0 FN 0 TN 64
TP 0 FP 0 FN 0 TN 64
TP 0 FP 0 FN 0 TN 64
TP 0 FP 0 FN 0 TN 64
TP 0 FP 0 FN 0 TN 64
TP 0 FP 64 FN 0 TN 0
TP 0 FP 0 FN 0 TN 64
TP 0 FP 7 FN 0 TN 57
TP 0 FP 59 FN 0 TN 5
 f1 score .... [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
# predict the whole test set 
data_directory = './DISFA_fold/'

npz_files = [f for f in os.listdir(data_directory) if f.startswith('DISFA_fold0')]
y_pred_acc = []
y_pred_f1 = np.array([])
y_true_ex = []

for file_name in npz_files:
    data_test = data_directory + file_name
    print('Predicting ' + data_test)

    # Load data from .npz file
    x = np.load(data_test)
    x_train1 = x['x_train1']
    x_train1 = np.concatenate((x_train1, x_train1, x_train1), axis=-1)
    y_test = x['y_train']
    y_test = y_test[:, ind1]

    # Predict using the model
    y_predict = model.predict(x_train1)
    y_pred = y_predict[1]

    # Store predictions and ground truth for each file
    y_pred_acc.append(y_pred)
    y_true_ex.append(y_test)

# Concatenate predictions and ground truth arrays
y_pred_acc = np.vstack(y_pred_acc)
y_true_ex = np.vstack(y_true_ex)

In [ ]:
att_f1 = []
for l in range(AU_count):
    a = np.multiply(y_true_ex[:, l], 1)
    b = np.multiply(y_pred_acc[:, l] > 0.5, 1)
    TP = sum(np.multiply(np.logical_and(a == 1, b == 1), 1))
    FP = sum(np.multiply(np.logical_and(a == 0, b == 1), 1))
    FN = sum(np.multiply(np.logical_and(a == 1, b == 0), 1))
    TN = sum(np.multiply(np.logical_and(a == 0, b == 0), 1))
    Precision = TP / (TP + FP + 0.00001)
    print('TP',TP,'FP',FP,'FN',FN,'TN',TN)
    Recall = TP / (TP + FN + 0.00001)
    att_f1.append(2 * Precision * Recall / (Precision + Recall + 0.000001))
print(' f1 score ....', att_f1)

TP 0 FP 3 FN 0 TN 24189
TP 0 FP 0 FN 0 TN 24192
TP 0 FP 13 FN 0 TN 19334
TP 0 FP 0 FN 0 TN 24192
TP 0 FP 8931 FN 0 TN 15261
TP 0 FP 202 FN 0 TN 23990
TP 4252 FP 2059 FN 592 TN 12444
TP 0 FP 0 FN 0 TN 24192
TP 0 FP 24192 FN 0 TN 0
TP 0 FP 16 FN 0 TN 24176
TP 0 FP 5469 FN 0 TN 13878
TP 3541 FP 6223 FN 1303 TN 8280
 f1 score .... [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7623482298269417, 0.0, 0.0, 0.0, 0.0, 0.4848024038090206]


# 5. Loss Check


In [ ]:
trainer.history

{'loss': [0.3282294273376465],
 'att_outputs_loss': [0.9186989665031433],
 'per_outputs_12_loss': [0.35993966460227966],
 'att_loss_loss': [0.03427758440375328],
 'feat_outputs_12_loss': [2.910877650030841e-10],
 'att_outputs_macro_f1': [0.4184337556362152],
 'per_outputs_12_macro_f1': [0.7316012978553772],
 'att_loss_huber_loss': [0.03427761793136597],
 'val_loss': [1.0279409885406494],
 'val_att_outputs_loss': [0.6830723881721497],
 'val_per_outputs_12_loss': [3.0810048580169678],
 'val_att_loss_loss': [0.033783432096242905],
 'val_feat_outputs_12_loss': [0.313903272151947],
 'val_att_outputs_macro_f1': [0.5313493013381958],
 'val_per_outputs_12_macro_f1': [0.683148980140686],
 'val_att_loss_huber_loss': [0.03378342092037201],
 'lr': [1e-05]}